In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
username='email'
password='password' #oculté mis credenciales reales

In [3]:
df_stations=pd.read_csv('data/d07_text_meta_2023_12_22.txt', delimiter='\t')
df_stations.head(10)

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,715898,5,N,7,37,40032.0,.68,117.313,33.880069,-118.021261,0.828,ML,3,PHOEBE,2029,NaN,NaN,NaN
1,715900,5,S,7,37,40032.0,1.06,117.630,33.882892,-118.026822,NaN,OR,1,VALLEY VIEW,3255,NaN,NaN,NaN
2,715901,5,N,7,37,40032.0,1.11,117.743,33.883400,-118.027451,NaN,OR,1,VALLEY VIEW,3268,NaN,NaN,NaN
3,715903,5,N,7,37,69154.0,1.56,118.193,33.886992,-118.034125,NaN,OR,1,ALONDRA,3269,NaN,NaN,NaN
4,715904,5,S,7,37,69154.0,2.27,118.840,33.892489,-118.044573,NaN,OR,1,CARMENITA,3253,NaN,NaN,NaN
5,715905,5,N,7,37,52526.0,2.52,119.153,33.894587,-118.048098,NaN,OR,1,CARMENITA,3270,NaN,NaN,NaN
6,715906,5,S,7,37,52526.0,3.28,119.850,33.900669,-118.059422,NaN,OR,1,ROSECRANS,3252,NaN,NaN,NaN
7,715907,5,S,7,37,52526.0,3.62,120.190,33.903961,-118.064046,NaN,OR,1,FIRESTONE,3251,NaN,NaN,NaN
8,715908,5,S,7,37,52526.0,4.28,120.850,33.911074,-118.071686,NaN,OR,1,SAN ANTONIO/NORWALK,3090,NaN,NaN,NaN
9,715910,5,N,7,37,52526.0,4.58,121.213,33.913564,-118.075867,NaN,OR,1,NORWALK SB,3273,NaN,NaN,NaN


In [6]:
df_stations_util=df_stations[['ID','Fwy','Dir']].copy()
df_stations_util

,ID,Fwy,Dir
0,715898,5,N
1,715900,5,S
2,715901,5,N
3,715903,5,N
4,715904,5,S
...,...,...,...
4883,777794,5,S
4884,777795,5,N
4885,777796,5,N
4886,777797,5,S


In [7]:
login_url = "https://pems.dot.ca.gov/?dnode=Login"
payload = {
    'username': username,
    'password': password
}
session = requests.Session()
r = session.post(login_url, data=payload)
if "logout" not in r.text.lower():
    print("⚠️ Login posiblemente falló, revisa credenciales.")
else:
    print("✅ Sesión iniciada correctamente")

✅ Sesión iniciada correctamente


In [11]:
rows = []
for _, st in df_stations_util.iterrows():
    freeway = st['Fwy']
    direction = st['Dir']
    sid = st['ID']

    url = f"https://pems.dot.ca.gov/?fwy={freeway}&dir={direction}&station_id={sid}&dnode=VDS"
    resp = session.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Localizar la tabla con Roadway Information
    tables = soup.find_all('table', class_='blue_outline_table')
    design_speed = None
    for table in tables:
        caption = table.find('td', class_='blue_outline_caption')
        if caption and "Roadway Information" in caption.text:
            trs = table.find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                if len(tds) == 2 and 'Design Speed Limit' in tds[0].get_text(strip=True):
                    design_speed = tds[1].get_text(strip=True)
                    break

    rows.append({
        "freeway": freeway,
        "direction": direction,
        "station_id": sid,
        "design_speed_limit": design_speed
    })

    print(f"{sid}: {design_speed}")
    time.sleep(1)


715898: 70 mph
715900: 70 mph
715901: 70 mph
715903: 70 mph
715904: 70 mph
715905: 70 mph
715906: 70 mph
715907: 70 mph
715908: 70 mph
715910: 70 mph
715911: 70 mph
715912: 70 mph
715913: 70 mph
715915: 70 mph
715916: 70 mph
715918: 70 mph
715920: 70 mph
715923: 70 mph
715924: 70 mph
715925: 70 mph
715926: 70 mph
715927: 70 mph
715928: 70 mph
715929: 70 mph
715930: 70 mph
715932: 70 mph
715933: 70 mph
715935: 70 mph
715937: 70 mph
715938: 70 mph
715941: 70 mph
715944: 70 mph
715947: 70 mph
715949: 70 mph
715950: 70 mph
715952: 70 mph
715953: 70 mph
715957: 70 mph
715958: 70 mph
715959: 70 mph
715961: 70 mph
715963: 70 mph
715964: 70 mph
715966: 70 mph
715967: 70 mph
715968: 70 mph
715969: 70 mph
715970: 70 mph
715971: 70 mph
715972: 70 mph
715973: 70 mph
715974: 70 mph
715975: 70 mph
715976: 70 mph
715977: 70 mph
715978: 70 mph
715979: 70 mph
715980: 70 mph
715981: 70 mph
715982: 70 mph
715983: 70 mph
715985: 70 mph
715986: 70 mph
715987: 70 mph
715988: 70 mph
715989: 70 mph
715990: 70

KeyboardInterrupt: 

In [12]:
rows = []
for _, st in df_stations_util.iterrows():
    freeway = st['Fwy']
    direction = st['Dir']
    sid = st['ID']

    url = f"https://pems.dot.ca.gov/?fwy={freeway}&dir={direction}&station_id={sid}&dnode=VDS"
    resp = session.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Localizar la tabla con Roadway Information
    tables = soup.find_all('table', class_='blue_outline_table')
    design_speed = None
    for table in tables:
        caption = table.find('td', class_='blue_outline_caption')
        if caption and "Roadway Information" in caption.text:
            trs = table.find_all('tr')
            for tr in trs:
                tds = tr.find_all('td')
                if len(tds) == 2 and 'Design Speed Limit' in tds[0].get_text(strip=True):
                    design_speed = tds[1].get_text(strip=True)
                    break

    rows.append({
        "freeway": freeway,
        "direction": direction,
        "station_id": sid,
        "design_speed_limit": design_speed
    })

    print(f"{sid}: {design_speed}")
    time.sleep(1)

715898: 70 mph
715900: 70 mph
715901: 70 mph
715903: 70 mph
715904: 70 mph
715905: 70 mph
715906: 70 mph
715907: 70 mph


KeyboardInterrupt: 